In [ ]:
import pandas as pd
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
import os
import random
import seaborn as sns
from keras.models import Sequential
from keras import optimizers
from keras import backend as B
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
directory = "../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/annotations"
imagedirectory = "../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images"
df = pd.read_csv("../input/face-mask-detection-dataset/train.csv")
df_test = pd.read_csv("../input/face-mask-detection-dataset/submission.csv")

In [ ]:
# The creation of helper functions would be necessary task to refine and prepare the data:
cvv = cv2.dnn.readNetFromCaffe('weights.caffemodel')         #This function is used to read a deep learning network represented in the Caffe framework's format..the pathway should be actual to file in the .caffemodel

In [ ]:
def JSONpath(filePathandName):
    with open(filePathandName,'r') as f:
        return json.load(f)    #The JSONpath function retrieves the json file containing the bounding box data in the training dataset.
def adjust_gamma(image, gamma=1.0):
    invertGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invertGamma) * 255 for i in np.arange(0, 256)])
    return cv2.LUT(image.astype(np.uint8), table.astype(np.uint8))        #The adjust_gamma function is a non-linear operation used to encode and decode luminance or tristimulus values in video or still image systems.If gamma <1, the image will shift to the darker end of the spectrum and when gamma> 1, there will be more light in the image.

In [ ]:
jsonfile = []
for i in os.listdir(directory):
  jsonfile.append(JSONpath(os.path.join(directory,i)))
jsonfile[0]   #creating a list of JSON file paths, and then accessing the first JSON file path in the list.

In [ ]:
df = pd.read_csv("train.csv")
df.head()

In [ ]:
#Using the mask and the non_mask labels,
#the bounding box data of the json files is extracted. The faces of a particular image are extracted and stored in the data list with its tag for the learning process.
#iterating over unique values in a column named "name" in a DataFrame (df). For each unique value,constructing a JSON file name by appending ".json" to it. Then,retrieving annotations from the JSON file using a function called JSONpath() and filtering based on the "classname" attribute. If the "classname" is in the "mask" list,extracting a bounding box and corresponding image from another directory using OpenCV (cv2).resizing the image to a specified size (img_size) and then appending the resized image and its label to a data list.
data = []
imgsize = 124
mask = ['face_with_mask']
non_mask = ['face_no_mask']
labels = {'mask' : 0, 'without mask' : 1}
for i in df["name"].unique():
  f = i + ".json"
  for j in JSONpath(os.path.join(directory,f)).get("Annotations"):
    if j["classname"] in mask:
      a,b,c,d = j["BoundingBox"]
      img = cv2.imread(os.path.join(imagedirectory,i),1)
      img = img[b:d,a:c]
      img = cv2.resize(img,(imgsize,imgsize))
      data.append([img,labels["mask"]])

      #the classname is not in the "non_mask" list. When this condition is met,extract the bounding box coordinates from the annotation and use OpenCV to read, crop, and resize the corresponding image. Then,append the resized image along with its label to the "data" list. Finally, you shuffle the data.
    if j["classname"] in non_mask:
      a,b,c,d = j["BoundingBox"]
      img = cv2.imread(os.path.join(imagedirectory,i),1)
      img = img[b:d,a:c]
      img = cv2.resize(img,(imgsize,imgsize))
      data.append([img,labels["without mask"]])
random.shuffle(data)

In [ ]:
plot = []     #assign an empty list named as plot
for face in data:     #iterate over the dat for the faces with mask...if the face selected has value as 0 then masked else non masked..construct the plot of the data stored in the list plot
    if(face[1] == 0):
        plot.append("Mask")
    else:
        plot.append("No Mask")
sns.countplot(plot)

In [ ]:
#In any different model training we have to balanced out the different features used but here since we are usingssd model(vgg16),it doesn't matter much
X = []
Y = []
for features,label in data:
    X.append(features)
    Y.append(label)

X = np.array(X)/255.0
X = X.reshape(-1,124,124,3)
Y = np.array(Y)

In [ ]:
#Model Building
model = Sequential()
model.add(Conv2D(32,(3,3),padding = "same",activation = "relu",input_shape = (124,124,3)))
model.add(Conv2D(64,(3,3),activation = "relu"))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = "rmsprop"loss='binary_crossentropy', optimizer='adam' ,metrics=['accuracy'])

In [ ]:
#Model Training
x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size = 0.9,random = 123)
datagenerator = ImageDataGenerator(featurewise_center=False,samplewise_std_normalization=False,zca_whitening=False,rotation_range=35,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,vertical_flip=False)
datagenerator.fit(x_train)
modelfit = model.fit_generator(datagenerator.flow(x_train,y_train,batch_size = 32),steps_per_epoch = x_train.shape[0]//32,epochs = 50,verbose = 1,validation_data = (x_test,y_test))


In [ ]:
testing = ['1114.png','1554.jpg', '1072.jpg','0012.jpg','0357.jpg','1374.jpg','1234.jpg','1323.jpg']
gamma = 2
figure = plt.figure((figsize = (14,14)))
rows = 4
cols = 2
axes = []
assigned = {'0' : 'MASK', '1' : 'NON MASK'}
for j,m in enumerate(testing):
  image =  cv2.imread(os.path.join(imagedirectory,m),1)
  image =  adjust_gamma(image, gamma=gamma)
  (c, d) = image.shape[:2]
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300,300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
  cvv.setInput(blob)
  detections = cvv.forward()

#For each image filename in the testing list:read the image using OpenCV (cv2.imread()),adjust the gamma of the image using a function named adjust_gamma().resize the image to a size of 300x300 pixels and convert it into a blob suitable for input to a neural network using OpenCV's cv2.dnn.blobFromImage() function.set the input of the neural network (cvNet.setInput(blob)).perform forward pass inference on the neural network (cvNet.forward()), presumably for object detection.

In [ ]:
for i in range(0,detections.shape[2]):
  try:
    boxi = detections[0,0,i,3:7] * np.array([d,c,d,c])
    (startX, startY, endX, endY) = boxi.astype("int")
    frame = image[startY:endY, startX:endX]
    confidence = detections[0, 0, i, 2]
    if confidence > 0.2:
      m = cv2.resize(frame(imgsize,imgsize))
      m = np.array(m) / 255.0
      m = m.reshape(1,124,124,3)
      result = model.predict(m)
      if result > 0.5:
        label_Y = 1
      else:
        label_Y = 0
cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
cv2.putText(image,assigned[str(label_Y)] , (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (36,255,12), 2)
  except:pass
axes.append(figure.add_subplot(rows,cols,j + 1))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('on')
plt.gca().add_patch(plt.Rectangle((boxi[0], boxi[1]), boxi[2] - boxi[0], boxi[3] - boxi[1], fill=False, edgecolor='green', linewidth=6))
plt.show()
#Bounding box coordinates and confidence score are extracted.
#If the confidence score exceeds a threshold (0.2), further processing occurs:
#The detected object is resized to a specified size.
#Normalization and reshaping are applied for prediction.
#Based on the prediction result, a bounding box and label are drawn on the original image.
#The original image with overlaid bounding boxes is visualized using Matplotlib.